<table class="buttons" align="center">
    <td>
        <a target="_blank" href="https://colab.research.google.com/github/dl-ub-summer-school/2019/blob/master/seminar4.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Google Colab дээр нээх</a>
    </td>
    <td>
        <a target="_blank" href="https://github.com/dl-ub-summer-school/2019/blob/master/seminar4.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />GitHub дээр нээх</a>
    </td>
    <td>
        <a target="_blank" href="https://sites.google.com/view/dlub/dl-ub-2019"><img src="https://avatars0.githubusercontent.com/u/52651086?s=32&v=4">Зуны сургалтын вебсайт</a>
    </td>
</table>


#Translation with a Sequence to Sequence Network and Attention
*************************************************************
**Author**: `Sean Robertson <https://github.com/spro/practical-pytorch>` \
Эх сурвалж colab \
https://colab.research.google.com/drive/1wS4qI40tT0COGgSKo_oMas_4h6OLBjD2#scrollTo=eN8wqpjCdaoG&line=5&uniqifier=1 \

Энэхүү материалыг Sean Robertson -ийн эх кодыг монгол болон англи хэлэнд зориулан бэлтгэсэн ба семинарын үндсэн сэдэв зориулан зарим хэсгүүдэд нэмэлт тайлбар зүүн, зарим хэсгүүдийг нь дарсан болно.

::

    [KEY: > input, = target, < output]

    > Тэд ЭЗЭНий тухай худал хэлж, “Энэ нь Тэр биш. Золгүй явдал бидэн дээр ирэхгүй. Бид өлсгөлөн, эсвэл илдийг үзэхгүй.
    = They have belied the LORD, and said, It is not he; neither shall evil come upon us; neither shall we see sword nor famine:
    < And they shall not shall the the the the the the the the the the LORD, and the the the the the of the the <EOS>

    > Өшөө авах хилэн гарч ирэхээр, Би түүний цусыг далдлуулахгүйн тулд нүцгэн хадан дээр урсгасан”.
    = That it might cause fury to come up to take vengeance; I have set her blood upon the top of a rock, that it should not be covered.
    < And I will bring forth to the of the I the of the the the I the of the the the <EOS>
    
**Анхаарах зүйлс:**

Эхлээд дараах үйлдлүүдийг дагаж хийгээд энэ нөтбүкийг өөрийн Драйвтаа хадгалж аваарай.

1. Зүүн дээд буланд байгаа _File_ дээр дараад
2. _Save copy in Drive_ дээр дарж өөрийн хувийг үүсгээд
3. Үүсгэсэн Колаб нөтбүк дээрээ ажилла.
4. Дараа нь энэ Колаб нөтбүкээ Драйв доторх Colab Notebooks гэдэг нэртэй, автоматаар үүсдэг хавтсан дотроос олж үзээрэй.




**Хэрэгтэй сангуудаа суулгах нь**

In [0]:
%matplotlib inline

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [0]:
print(device)

cuda


In [0]:
!git clone --recursive https://github.com/dl-ub-summer-school/2019.git

Cloning into '2019'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 58 (delta 27), reused 22 (delta 7), pack-reused 0
Unpacking objects: 100% (58/58), done.


Дата бэлтгэл
==================
NLP -ийн аливаа таскыг хийхэд юун түрүүнд өгөгдлөө машинд илэрхийлэхэд тохиромжтой хэлбэр рүү шилжүүлэх хэрэгтэй.
Ийн шижүүлэхдээ data preprocessing гэх хэд хэдэн алхамтай үйлдлийг дамждаг. 
Семинарын цагтаа тааруулан бид тэдгээр шатуудын ихэнхийг алгасан цэвэрхэн датан дээр ажиллах болно.

*  Parallel corpora format
> Machine translation гэх NLP таскын датаг ийн нэрлэдэг. Семинартаа зориулан сонгож уг форматаар бэлдсэн дата - Библи
*  Датаг python дээр унших
> Датаны нэр - eng-mng.txt

*Дата сонголтын тухай*:
*(Шашны урсгалын судар тул орчуулгын хувьд харьцангуй алдаа багатай)*



**Word embedding**

Word embedding хийх олон арга байдаг талаар эхний лекцэн дээр дурьдсан.

Энэ удаад Embedding хийхэд хамгийн энгийн арга буюу нийт текстийг хоосон зайгаар нь салгаж үг бүрийг нэг токен гэж үзэцгээе. Үүний дараа токен бүрийг тодорхой урттай вектороор илэрхийлэнэ.
 



##Дата бэлтгэлийн функцууд

Доорхи класст ялгаатай токенуудыг багтаасан dictionary үүсгэж байна. 

In [0]:
# SOS - Start of sequence token
# EOS - End of sequence token

SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Гараас өгөх текстүүд нь Unicode форматтай байгаа ба дараах үйлдлүүдийг хийх функцууд.

> 1. ASCII формат руу хөрвүүлэх.
> 2. Нормалчлах буюу: 
>> 2.1.Том үсгүүдийг жижиг үсэг болгох (lowercase) \
>> 2.2. " . ", " ! ", " ? " тэмдэгтүүдийг солино.





In [0]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    return s

Орчуулгын таскад бэлдсэн датаны мөр бүрийг pair (хосмог гэх) болгон бэлтгэх функц.

In [0]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Файлыг уншин мөр бүрт шилжүүлж байна.

    
    input_data='/content/2019/seminar4/%s-%s.txt'
    lines = open(input_data % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')
    
#     Өгөгдсөн дата файлыг мөр бүрийг python list хэлбэрт шилжүүлэн, python list-эд хадгалж байна. 
#     Ө.х.: листийг листэнд хадгалж байна.

    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    
#     Хоёр хэлийг бэлтгэгдсэн датанаас урвуу чиглэлд орчуулалт сургах тохиолдолд уг flag-ийг ашиглах.
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Sequence нь өгөгдсөн MAX_LENGTH -ээс бага урттай байдаг sample -уудыг ялгаж авна.
Өөрөөр хэлбэл:

> MAX_LENGTH = 5 үед \
> **In the beginning God created the heaven and the earth.	  Эхэнд Бурхан тэнгэр газрыг бүтээжээ.**
> гэдэг өгүүлбэрүүд сургалтанд орохгүй.


Энэ семинарын хувьд, нийт текст дээрээ дата анализ хийн sequence бүрийн токены тоог гарган дундажлан 25 гэж тогтсон. Тиймээс энэ хэмжээгээр явах юм.

In [0]:
MAX_LENGTH = 25

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

##Дата бэлтгэлийн гол функц

1.   Оролтын файлаа кодондоо уншиж мөр бүрийг нь орчуулгын хосмог болгоно.
2.   Текстээ нормалчилана.
3.    Өгүүлбэрийн хосмогуудаасаа үгийн жагсаалт гаргана.




In [0]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted unique words of", input_lang.name, "is", input_lang.n_words)
    print("Counted unique words of", output_lang.name, "is", output_lang.n_words)
    print("Total words of source language %s" % input_lang.name, sum(list((input_lang.word2count).values())))
    print("Total words of target language %s" % output_lang.name, sum(list((output_lang.word2count).values())))
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'mng', True)
print(random.choice(pairs))

Reading lines...
Read 31103 sentence pairs
Trimmed to 15283 sentence pairs
Counting words...
Counted words:
mng 27640
eng 15352
Total words of source language mng 212607
Total words of target language eng 265674
['учир нь минии эцэг та нарын төлөө тулалдаж, өөриин амииг үл хаирлан та нарыг мидианы савраас аварсан билээ .', '(for my father fought for you, and adventured his life far, and delivered you out of the hand of midian:']


The Seq2Seq Model
=================

Recurrent Neural Network буюу RNN нь дараалал хэлбэртэй датан дээр ажилладаг. \
Seq2Seq буюу Sequence to Sequence модел нь encoder-decoder гэсэн хоёр RNN архитектурыг ашигладаг. Encoder нь оролт sequence-ийг тодорхой урттай вектор руу буулгахад, decoder нь уг буусан векторыг гаралт sequence руу хөрвүүлдэг. Энэхүү дундын векторын (цаашид дундын вектор гэх) үүрэг нь оролтын датаны **бүрэн мэдээллийг агуулан цааш дамжуулах**  ба моделийн давуу тал нь энгийн RNN -шиг оролт болон гаралтын хэмжээнүүд нь нэг нөгөөгөөсөө шууд хамааралгүй байна.




Энкодер
-----------

Sequence-ийн токен бүрт тодорхой hidden_size урттай вектор гарах ба  **1 x hidden_size**   (1, 256) хэмжээтэй матриц байна гэж төсөөлж болно.




In [0]:
class EncoderRNN(nn.Module):
  
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Декодер
-----------

Энкодер -оос гарч ирсэн дундын векторыг орчуулах хэлний sequence рүү буулгана. 




**Энгийн декодер** 


In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

**Attention Decoder**

Attention mechanism нь дэкодэрийг алхам бүртээ энкодероос гарч буй векторуудын өөр өөр хэсгүүдэд төвлөрөх нөхцөлийг бүрдүүлдэг. Ингэхдээ


> 1. *Attention weights* -үүдийг тооцон
> 2. Энэхүү жинг энкодерын гаралтын вектороор үржүүлэнэ. Ингэснээр оролтын sequence-ийн тодорхой хэсгийн мэдээллийг агуулсан вектор гарах юм.


 *Attention weights* -үүдийг тооцоолохдоо декодерийн оролт болон hidden state -үүдийг нь оролтоор авсан feed-forward layer явуулана. Өгөгдөл текстүүд маань янз бүрийн урттай байдаг. Харин цаана явагдах тооцооллууд тогтмол хэмжээтэй байх хэрэгтэй тул sequence болон үгсийн жагсаалтыг гаргадаг.
 
 


In [0]:
# Энд "---"-аар тухайн үйлдлийн гаралтуудын хэмжээг тавьлаа.

class AttnDecoderRNN(nn.Module):
  
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
      
        super(AttnDecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size # given as 256
        self.output_size = output_size # target_language_dict_size
        self.dropout_p = dropout_p # given as 0.1
        self.max_length = max_length # given as 25

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        # ---1x1x256
        embedded = self.embedding(input).view(1, 1, -1) 
        
        # ---1x1x256
        embedded = self.dropout(embedded) 
        
        # --- 1x512 
        concated=torch.cat((embedded[0], hidden[0]), 1)
        
        # --- 1x25
        attention=self.attn(concated)
        
        # --- 1x25
        attn_weights = F.softmax(attention, dim=1)
        
        # --- 1x1x25 
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), # --- 1x1x25
                                 encoder_outputs.unsqueeze(0)) # --- 1x25x256
        
        # --- 1x512
        output = torch.cat((embedded[0], attn_applied[0]), 1) 
        
        # --- 1x1x256
        output = self.attn_combine(output).unsqueeze(0)
        
        # --- 1x1x256
        output = F.relu(output)
        
        # 1x1x256, 1x1x256
        output, hidden = self.gru(output, hidden) 
        
        #1x{target_language_dict_size}
        output = F.log_softmax(self.out(output[0]), dim=1) 
        
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Энд,  "local attention"-ий тухай ойлголтыг `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>` -ээс уншиж болно.

Сургалтын датаг бэлтгэх
-------------------------------
Модел руу датаг оруулахдаа тэнсор хэлбэр рүү шилжүүлэнэ. Ингэхдээ дээр зарласан Lang class -аас индексээ аван дараах байдлаар үүсгэнэ.


In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Модел сургалт
------------------
Дараах алхамуудын хийнэ:

-  Цагаа эхлүүлэх
-  Optimizers and criterion-үүдэд анхны утга өгөх
-  Сургалтын датагаа хосмог болгон бэлдэх
-  Start empty losses array for plotting





Сургалтын явцыг мэдэхэд зориулсан нэмэлт функц




In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

Сургалтын нэг алхамыг хийж буй функц.

Энд, нэг алхам гэдэг нь сургалтын бүх датагаа нэг удаа харахыг хэлнэ.

In [0]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad() 
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0) #column number
    target_length = target_tensor.size(0) #column number

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
        
    # initialize decoder
    
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden


    for di in range(target_length):
        
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        
        topv, topi = decoder_output.topk(1) #target word-ийг сонгож байгаа нь.
        decoder_input = topi.squeeze().detach()  # дараагын токены индексыг сонгож байгаа нь.

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
            break

    loss.backward() # backpropagation тооцож байгаа нь

    encoder_optimizer.step() #weight -үүдээ шинэчлэж байгаа нь
    decoder_optimizer.step() #weight -үүдээ шинэчлэж байгаа нь

    return loss.item() / target_length

Бид Stochastic Gradient Descent ашиглаж байгаа учраас датаны нэг sample бүр дээр лосс-оо тооцон weight-үүдээ шинэчлэнэ. Тиймээс дээр тодорхойлсон функцыг алхамыг хэдэн ч удаа хийсэн болно.




In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss() #negative log likelihood loss

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Графикаар үзэх
----------------

Лoss -ийн утгуудыг хадгалж аван ``plot_losses``, үүнийгээ харах функцы




In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

##Модел шалгалт


Модел сургалттай адил үйл явцтай авч датаны хувьд орчуулах хэлэн дээрхи дата нь орно. Өөрөөр хэлбэл, тухайн нэг өгүүлбэр ороход декодер орчуулан үг бүрээр нь таамаглал хийн гаргах болно.




In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Сургалтыг эхлүүлэх ба сургаснаа шалгах
==================================

Hidden node -ийн утгыг гараас өгөн сургалтыг эхлүүлж болно.





In [0]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 20000, print_every=4000)

2m 35s (- 10m 20s) (4000 20%) 4.8725
5m 11s (- 7m 47s) (8000 40%) 4.8061
7m 55s (- 5m 16s) (12000 60%) 4.7652
10m 40s (- 2m 40s) (16000 80%) 4.7310
13m 24s (- 0m 0s) (20000 100%) 4.6628


In [0]:
evaluateRandomly(encoder1, attn_decoder1)

> нэг үеиинхэн таны үилсииг нөгөө үеиинхэнд магтаж, таны хүчирхэг үилдлүүдииг тунхаглана .
= one generation shall praise thy works to another, and shall declare thy mighty acts .
< and the the one the the and the and the the . . <EOS>

> шударга хиигээд зөвииг үилдэх нь эзэнд өргөлөөс ч илүү таалагддаг .
= to do justice and judgment is more acceptable to the lord than sacrifice .
< the the the the the the the the the the the the the the <EOS>

> эзэн есүсииг амилуулсан нэгэн биднииг ч бас есүстэи хамт амилуулж, биднииг та нарын хамт өөриинхөө өмнө аваачна гэдгииг бид мэддэг .
= knowing that he which raised up the lord jesus shall raise up us also by jesus, and shall present us with you .
< and the lord the the the the the the the the the the the the the the . the <EOS>

> паул ч, апол ч, кеф ч, ертөнц ч, амь хиигээд үхэл ч, одоо хиигээд ирээдүи ч бүгд та нарынх аж .
= whether paul, or apollos, or cephas, or the world, or life, or death, or things present, or things to come; all are you

Attention-г харах
---------------------

Өгүүлбэрийн хаана attend хийж байгааг харах функц.




In [0]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "эзэний тухай худал цуу яриа гарчээ") 
plt.matshow(attentions.numpy())

For a better viewing experience we will do the extra work of adding axes
and labels:




In [0]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("эзэний тухай худал цуу яриа гарчээ") 


## Нэмэлт санал болгох материалууд

-`Sequence to sequence network <http://arxiv.org/abs/1409.3215>`__ 
- `Attention mechanism <https://arxiv.org/abs/1409.0473>`__ \
-  `Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation <http://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`
- `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`  
- `Encoder Decoder network <https://arxiv.org/pdf/1406.1078v3.pdf>`